In [27]:
import json
import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import datetime
def clean_data(file,options,stime='09:30:00',etime='16:00:00'):
        #read file
        df = pd.read_csv(file+".csv",sep=';',skiprows=3)
        #remove unwanted rows
        df = df.iloc[:,:-1]
        #read time  and date then make time as index
        df['start']=pd.to_datetime(df['Date/Time'],format='%m/%d/%y, %I:%M %p')
        df=df.set_index(['Id']).rename(columns={'Price':'open_trade'})
        #drop Id and unnamed column
        df.drop(["P/L","Trade P/L","Side","Date/Time"],axis=1,inplace=True)
        # drop last n rows
        df.drop(df.tail(3).index,inplace=True)
        #add open trade and remove unwanted symbols
        df['open_trade'] = df['open_trade'].map(lambda x: x.lstrip('($').rstrip(')')).str.replace(',','')
        #add close time of trade to same row of start time by shifting start row
        df['end']=df["start"].shift(-1, axis = 0)
        #add duration column
        df['duration']=df.end-df.start
        #add close price of trade to same row of start price by shifting start row
        df['close_trade']=df["open_trade"].shift(-1, axis = 0)
        #convert to numers
        df['open_trade']=pd.to_numeric(df['open_trade'])
        df['close_trade']=pd.to_numeric(df['close_trade'])
        #add profit column
        df['profit']=(df.close_trade-df.open_trade)*df.Amount
        #if options to add fees
        if options :
            fees=1.5*df.Quantity
            df['profit']=df['profit']-fees
        df['percent']=df.profit*100/(df.open_trade*df.Amount)
        df=df[::2].dropna()
        #add win column
        df['win'] = np.where(df['profit']>0, True, False)
        # set index
        df.set_index(['start'],inplace = True)
        #retrurn dataframe
        return df.between_time(stime,etime)
class Backtest:
    def __init__(self,file_name,options=False,stime='09:30:00',etime='16:00:00'):
        self.stime=stime
        self.etime=etime
        self.file_name=file_name
        self.ticker=self.file_name.split('-')[0]
        self.info=self.file_name.split('-')[1]
        self.timeframe=int(self.file_name.split('-')[2])
        self.range=int(self.file_name.split('-')[3])
        self.options=options
        self.df=clean_data(file=self.file_name,options=self.options,stime=self.stime,etime=self.etime)
        self.strategy=self.df['Strategy'].iloc[0]
        self.position_size=int(self.df['Amount'].iloc[0])
        self.total_profit=self.df['profit'].sum()
        self.total_trades=len(self.df[self.df['open_trade'] > 0])
        self.win_trades = len(self.df[self.df['profit'] > 0])
        self.loss_trades=len(self.df[self.df['profit'] < 0])
        self.breakeven=len(self.df[self.df['profit'] == 0])
        self.win_avg=self.df.groupby(['win']).mean().at[True,'profit']
        self.loss_avg=self.df.groupby(['win']).mean().at[False,'profit']
        self.reward_risk_ratio=abs(self.win_avg/self.loss_avg)
        self.win_ratio=self.win_trades/self.total_trades
        self.loss_ratio=self.loss_trades/self.total_trades
        self.expectency=self.total_profit/self.total_trades
    def __repr__(self):
        return __name__
    def result(self):
        dic={}
        dic['strategy']=self.strategy
        dic['ticker']=self.ticker
        dic['info']=self.info
        dic['timeframe']=self.timeframe
        dic['range']=self.range
        dic['position_size']=self.position_size
        dic['start_time']=self.stime
        dic['end_time']=self.etime
        dic['options']=self.options
        dic['total_profit']=self.total_profit
        dic['total_trades']=self.total_trades
        dic['win_trades']=self.win_trades
        dic['loss_trades']=self.loss_trades
        dic['breakeven']=self.breakeven
        dic['win_avg']=self.win_avg
        dic['loss_avg']=self.loss_avg
        dic['reward_risk_ratio']=self.reward_risk_ratio
        dic['win_ratio']=self.win_ratio
        dic['loss_ratio']=self.loss_ratio
        dic['expectency']=self.expectency
        return dic
    def json_result(self):
        return json.dumps(self.result())
    def print_unit(self,s,num,unit=''):
        print(s,'=',round(num, 2),unit)
    
    def report(self):
        print("Time:",self.stime,"-",self.etime)
        f_details=self.file_name.split('-')
        print("ticker:",f_details[0].upper()
              ,"\nTime frame:",f_details[1],"min\nDuration:",f_details[2],"Days")
        # Count number of True in the series
        print("Strategy: ",self.strategy)
        self.print_unit("Total profit",self.total_profit,"$")
        self.print_unit("Total trades",self.total_trades,"trades")
        print("No.Win trades=",self.win_trades)
        print("No.Loss trades=",self.loss_trades)
        print("No.BreakEven= ",self.breakeven,"it is equal",round(self.breakeven*100/self.total_trades, 2),"%")
        print("Expectency",round(self.total_profit/self.total_trades,4),"$")
        self.print_unit("Win Ratio",self.win_ratio*100,"%")
        self.print_unit("Loss Ratio",self.loss_ratio*100,"%")
        self.print_unit("Win average ",self.win_avg,"$")
        self.print_unit("Loss average ",self.loss_avg,"$")
        self.print_unit("Reward/Risk ratio",abs(self.reward_risk_ratio),"")
        self.print_unit("Average Profit/trade",self.total_profit/self.total_trades,"$")
    def comment(self):
        print("#Time:",self.stime,"-",self.etime)
        self.print_unit("#Total profit",self.total_profit,"$")
        self.print_unit("#Total trades",self.total_trades,"trades")
        print("#Expectency",round(self.total_profit/self.total_trades,4),"$")
        self.print_unit("#Win Ratio",self.win_ratio*100,"%")
        self.print_unit("#Loss Ratio",self.loss_ratio*100,"%")
        self.print_unit("#Win average ",self.win_avg,"$")
        self.print_unit("#Loss average ",self.loss_avg,"$")
        self.print_unit("#Reward/Risk ratio",abs(self.reward_risk_ratio),"")
        
        

In [28]:
filename=input("name: ")
print('#######')
Backtest(filename).comment()
print('#######')
Backtest(filename,0,'09:31:00','10:00:00').comment()

name:  aapl-ema9_sma200_atr0.5_onlytime_40-5-180


#######
#Time: 09:30:00 - 16:00:00
#Total profit = 12.32 $
#Total trades = 115 trades
#Expectency 0.1071 $
#Win Ratio = 47.83 %
#Loss Ratio = 52.17 %
#Win average  = 0.75 $
#Loss average  = -0.48 $
#Reward/Risk ratio = 1.56 
#######
#Time: 09:31:00 - 10:00:00
#Total profit = 12.32 $
#Total trades = 115 trades
#Expectency 0.1071 $
#Win Ratio = 47.83 %
#Loss Ratio = 52.17 %
#Win average  = 0.75 $
#Loss average  = -0.48 $
#Reward/Risk ratio = 1.56 


In [41]:
data=Backtest(filename).df.reset_index()

In [48]:
loss=data.loc[data['win']==False]

In [68]:
for index, row in loss.iterrows():
    print(row['start'],row['end'])

2020-10-26 09:35:00 2020-10-26 09:45:00
2020-10-29 09:35:00 2020-10-29 09:40:00
2020-11-02 09:35:00 2020-11-02 10:00:00
2020-11-04 09:35:00 2020-11-04 09:40:00
2020-11-04 09:45:00 2020-11-04 09:55:00
2020-11-09 09:35:00 2020-11-09 09:40:00
2020-11-10 09:35:00 2020-11-10 09:40:00
2020-11-12 09:35:00 2020-11-12 09:50:00
2020-11-13 09:35:00 2020-11-13 09:50:00
2020-11-16 09:45:00 2020-11-16 09:55:00
2020-11-18 09:40:00 2020-11-18 10:00:00
2020-11-27 09:35:00 2020-11-27 09:55:00
2020-12-03 09:35:00 2020-12-03 09:40:00
2020-12-17 09:35:00 2020-12-17 09:45:00
2020-12-29 09:35:00 2020-12-29 09:40:00
2020-12-29 09:45:00 2020-12-29 09:50:00
2021-01-08 09:35:00 2021-01-08 09:40:00
2021-01-08 09:45:00 2021-01-08 09:50:00
2021-01-13 09:35:00 2021-01-13 09:45:00
2021-01-15 09:35:00 2021-01-15 09:45:00
2021-01-19 09:35:00 2021-01-19 09:40:00
2021-01-22 09:45:00 2021-01-22 10:25:00
2021-01-25 09:35:00 2021-01-25 10:10:00
2021-01-26 09:35:00 2021-01-26 09:45:00
2021-01-27 09:45:00 2021-01-27 09:50:00


In [63]:
loss.iterrows()

<generator object DataFrame.iterrows at 0x7f3c5368a2e0>